<a href="https://colab.research.google.com/github/ankur-1232/Google-Colab-Notebooks/blob/main/Temporal_Feature_Engineering_and_Exploratory_Performance_Analysis_in_Formula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Temporal Feauture Engineering and Exploratory Performance Analysis in Formula 1**

Modern data science systems rarely operate on clean, single-table datasets.
Instead, they rely on **integrating multiple asynchronous data sources** sampled at different frequencies and latencies.

This project explores how **raw telemetry**, event-based, and contextual data from Formula 1 can be transformed into **reliable**, **time-aligned** feature tables suitable for exploratory analysis and downstream modeling.

The focus is not on predictive performance, but on data engineering correctness, feature validity, and temporal reasoning.

### Target Audience

*This notebook is designed for reviewers evaluating Data Science and Data Engineering internship candidates, with an emphasis on*:

* Handling messy, real-world data
* Temporal joins and feature synchronization
* Reproducible, modular data pipelines

### Data Source & Constraints
**Primary data source**: OpenF1 API (historical endpoints only)

**Data characteristics**:

* Multiple endpoints with different sampling frequencies
* Event-time timestamps (UTC)
* Partial and missing observations are expected

**Constraints**:

* No real-time streaming
* API responses may be empty or incomplete
* All joins must be time-aware

### Canonical Identifiers & Grain
Throughout this project, data will be indexed using the following identifiers:

* meeting_key

* session_key

* driver_number

* timestamp (UTC, ISO-8601)

The **canonical unit of analysis** will be:

**Driver x Session x Time**, with derived aggregations at the lap and stint level.

### Execution Philosophy
Development will proceed incrementally.
Each section will include explicit sanity checks before downstream dependencies are introduced.

## **Section 1** | **Environment Setup & API Utilities**

### 1.1 Environment & Library Setup

In [ ]:
# Core Python
import json
import time
from typing import Dict, Any, Optional

# Data handling
import pandas as pd
import numpy as np

# HTTP
import requests

# Display / debugging
from pprint import pprint

# Pandas display settings (for sanity)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("Environment ready")


Environment ready


### 1.2 Global Configuration

In [ ]:
# Base URL for OpenF1 API
OPENF1_BASE_URL = "https://api.openf1.org/v1"

# Default request timeout (seconds)
REQUEST_TIMEOUT = 10

# Small sleep to be polite to the API
REQUEST_SLEEP = 0.2

print("Config loaded")


Config loaded


### 1.3 Low-level API Fetch Utility

In [ ]:
def fetch_openf1_endpoint(
    endpoint: str,
    params: Optional[Dict[str, Any]] = None,
    sleep: float = REQUEST_SLEEP,
) -> pd.DataFrame:
    """
    Fetch data from an OpenF1 API endpoint and return a pandas DataFrame.

    Parameters
    ----------
    endpoint : str
        Endpoint name, e.g. 'sessions', 'laps', 'car_data'
    params : dict, optional
        Query parameters for filtering
    sleep : float
        Sleep duration after request (API politeness)

    Returns
    -------
    pd.DataFrame
        DataFrame containing the API response
    """

    url = f"{OPENF1_BASE_URL}/{endpoint}"

    try:
        response = requests.get(url, params=params, timeout=REQUEST_TIMEOUT)
    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Request failed for {endpoint}: {e}")

    if response.status_code != 200:
        raise RuntimeError(
            f"Non-200 response for {endpoint}: "
            f"status={response.status_code}, body={response.text[:200]}"
        )

    try:
        data = response.json()
    except json.JSONDecodeError:
        raise RuntimeError(f"Invalid JSON returned for {endpoint}")

    # Enforce expected structure
    if not isinstance(data, list):
        raise RuntimeError(
            f"Unexpected response structure for {endpoint}: {type(data)}"
        )

    df = pd.DataFrame(data)

    # Small pause to avoid hammering API
    time.sleep(sleep)

    return df


### 1.4 Connectivity & Schema Smoke Test

In [ ]:
# Fetch a very small, deterministic sample
sessions_df = fetch_openf1_endpoint(
    endpoint="sessions",
    params={"year": 2023}
)

print("Sessions shape:", sessions_df.shape)
sessions_df.head()


Sessions shape: (118, 14)


,session_key,session_type,session_name,date_start,date_end,meeting_key,circuit_key,circuit_short_name,country_key,country_code,country_name,location,gmt_offset,year
0,9222,Practice,Day 1,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,1140,63,Sakhir,36,BRN,Bahrain,Sakhir,03:00:00,2023
1,7763,Practice,Day 2,2023-02-24T07:00:00+00:00,2023-02-24T16:30:00+00:00,1140,63,Sakhir,36,BRN,Bahrain,Sakhir,03:00:00,2023
2,7764,Practice,Day 3,2023-02-25T07:00:00+00:00,2023-02-25T16:30:00+00:00,1140,63,Sakhir,36,BRN,Bahrain,Sakhir,03:00:00,2023
3,7765,Practice,Practice 1,2023-03-03T11:30:00+00:00,2023-03-03T12:30:00+00:00,1141,63,Sakhir,36,BRN,Bahrain,Sakhir,03:00:00,2023
4,7766,Practice,Practice 2,2023-03-03T15:00:00+00:00,2023-03-03T16:00:00+00:00,1141,63,Sakhir,36,BRN,Bahrain,Sakhir,03:00:00,2023


### 1.5 Sanity Checks

In [ ]:
# Basic sanity checks
assert not sessions_df.empty, "Sessions dataframe is empty"

expected_columns = {
    "session_key",
    "meeting_key",
    "session_name",
    "session_type",
    "date_start",
    "date_end",
}

missing = expected_columns - set(sessions_df.columns)
assert not missing, f"Missing expected columns: {missing}"

print("Sanity checks passed for sessions endpoint")


Sanity checks passed for sessions endpoint


In [ ]:
sessions_df.shape

(118, 14)

## **Section 2** | **Raw Data Ingestion**
**Goal**: Pull multiple OpenF1 endpoints into raw, untouched DataFrames and **understand their shape, sparsity, and time behavior** without joining anything.

### 2.1 Define Target Scope
* **Year**: 2023
* **Session type**: Race sessions only
* **Endpoints to ingest**:
    * sessions
    * drivers
    * laps
    * weather

In [ ]:
# Filter race sessions only
race_sessions_df = sessions_df[
    sessions_df["session_type"] == "Race"
].copy()

print("Race sessions shape:", race_sessions_df.shape)
race_sessions_df[["session_key", "meeting_key", "session_name", "country_name"]].head()


Race sessions shape: (29, 14)


,session_key,meeting_key,session_name,country_name
7,7953,1141,Race,Bahrain
12,7779,1142,Race,Saudi Arabia
17,7787,1143,Race,Australia
21,9069,1207,Sprint,Azerbaijan
22,9070,1207,Race,Azerbaijan


### 2.2 Sanity Check: Race Sessions

In [ ]:
assert not race_sessions_df.empty, "No race sessions found"
assert race_sessions_df["session_key"].is_unique, "Session keys should be unique"

print("Race sessions sanity checks passed")


Race sessions sanity checks passed


### 2.3 Ingest Drivers (Raw)

In [ ]:
drivers_raw = []

for session_key in race_sessions_df["session_key"].unique():
    df = fetch_openf1_endpoint(
        endpoint="drivers",
        params={"session_key": int(session_key)}
    )
    df["session_key"] = session_key
    drivers_raw.append(df)

drivers_df = pd.concat(drivers_raw, ignore_index=True)

print("Drivers shape:", drivers_df.shape)
drivers_df.head()


Drivers shape: (558, 12)


,meeting_key,session_key,driver_number,broadcast_name,full_name,name_acronym,team_name,team_colour,first_name,last_name,headshot_url,country_code
0,1141.0,7953,1.0,M VERSTAPPEN,Max VERSTAPPEN,VER,Red Bull Racing,3671C6,Max,Verstappen,https://www.formula1.com/content/dam/fom-websi...,NED
1,1141.0,7953,2.0,L SARGEANT,Logan SARGEANT,SAR,Williams,37BEDD,Logan,Sargeant,https://www.formula1.com/content/dam/fom-websi...,USA
2,1141.0,7953,4.0,L NORRIS,Lando NORRIS,NOR,McLaren,F58020,Lando,Norris,https://www.formula1.com/content/dam/fom-websi...,GBR
3,1141.0,7953,10.0,P GASLY,Pierre GASLY,GAS,Alpine,2293D1,Pierre,Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA
4,1141.0,7953,11.0,S PEREZ,Sergio PEREZ,PER,Red Bull Racing,3671C6,Sergio,Perez,https://www.formula1.com/content/dam/fom-websi...,MEX


### 2.5 Ingest Laps (Raw Controlled Sample)

In [ ]:
# Choose one race session for controlled ingestion
sample_session_key = int(race_sessions_df.iloc[0]["session_key"])
print("Sample session_key:", sample_session_key)

laps_df = fetch_openf1_endpoint(
    endpoint="laps",
    params={"session_key": sample_session_key}
)

print("Laps shape:", laps_df.shape)
laps_df.head()


Sample session_key: 7953
Laps shape: (1058, 16)


,meeting_key,session_key,driver_number,lap_number,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,is_pit_out_lap,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed
0,1141,7953,1,1,2023-03-05T15:03:38.500000+00:00,33.103,42.414,23.842,232.0,231.0,False,99.359,"[None, 2048, 2049, 2049, 2051, 2051, 2049, 204...","[2051, 2049, 2051, 2049, 2051, 2049, 2049, 204...","[2051, 2049, 2049, 2049, 2051, 2051]",252.0
1,1141,7953,11,1,2023-03-05T15:03:38.500000+00:00,34.120,43.216,24.069,230.0,237.0,False,101.405,"[None, 2048, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049]",252.0
2,1141,7953,16,1,2023-03-05T15:03:38.500000+00:00,33.938,42.549,24.122,226.0,250.0,False,100.609,"[None, 2048, 2049, 2051, 2049, 2049, 2049, 204...","[2049, 2051, 2049, 2049, 2049, 2049, 2049, 205...","[2049, 2049, 2049, 2049, 2049, 2049]",255.0
3,1141,7953,31,1,2023-03-05T15:03:38.500000+00:00,36.463,45.213,24.698,233.0,238.0,False,106.374,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]",254.0
4,1141,7953,27,1,2023-03-05T15:03:38.500000+00:00,37.038,45.924,24.741,220.0,239.0,False,107.703,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2048]",249.0


### 2.6 Laps Sanity Checks

In [ ]:
assert not laps_df.empty, "Laps dataframe is empty"

expected_lap_cols = {
    "driver_number",
    "lap_number",
    "lap_duration",
    "session_key",
}

missing = expected_lap_cols - set(laps_df.columns)
assert not missing, f"Missing expected lap columns: {missing}"

print("Laps sanity checks passed")


Laps sanity checks passed


## **Section 3** | **Canonical Keys & Time Normalization**
**Goal**: Make time a **first-class citizen** and ensure every dataset can be joined safely without leakage or ambiguity.

### 3.1 Timestamp Normalization
OpenF1 gives ISO timestamps, we enforce UTC-aware pandas datetime.

In [ ]:
def normalize_timestamps(df: pd.DataFrame, cols):
    """
    Convert specified columns to timezone-aware pandas datetime (UTC).
    """
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], utc=True, errors="coerce")
    return df


### 3.2 Apply to Known Time Columns

In [ ]:
# Normalize sessions timestamps
race_sessions_df = normalize_timestamps(
    race_sessions_df,
    ["date_start", "date_end"]
)

# Normalize laps timestamps
laps_df = normalize_timestamps(
    laps_df,
    ["date_start"]
)

print("Timestamp normalization complete")

Timestamp normalization complete


### 3.3 Timestamp Sanity Checks

In [ ]:
assert race_sessions_df["date_start"].dt.tz is not None
assert laps_df["date_start"].dt.tz is not None

print("Timezone sanity checks passed")

Timezone sanity checks passed


### 3.4 Define Canonical Event Time Column
In all downstream processing, **date_start** will be treated as the **event_time** for:
* Laps
* Telemetry
Aggregations(lap, stint, session) will be derived from this reference

### 3.5 Inspect Temporal Coverage

In [ ]:
laps_df["date_start"].describe()

,date_start
count,1058
mean,2023-03-05 15:48:52.929934848+00:00
min,2023-03-05 15:03:38.500000+00:00
25%,2023-03-05 15:25:23.323749888+00:00
50%,2023-03-05 15:48:22.483000064+00:00
75%,2023-03-05 16:12:17.605250048+00:00
max,2023-03-05 16:37:28.273000+00:00


In [ ]:
laps_df["date_start"].describe()


,date_start
count,1058
mean,2023-03-05 15:48:52.929934848+00:00
min,2023-03-05 15:03:38.500000+00:00
25%,2023-03-05 15:25:23.323749888+00:00
50%,2023-03-05 15:48:22.483000064+00:00
75%,2023-03-05 16:12:17.605250048+00:00
max,2023-03-05 16:37:28.273000+00:00


## **Section 4** | **Weather Data Ingestion & Temporal Resampling**
**Goal**: Bring in **continuous, time sampling data** and align it with **event-based lap data** without being leaking future information.

### 4.1 Ingest Weather Data (Raw)
Weather is session-scoped and time-sampled

In [ ]:
weather_df = fetch_openf1_endpoint(
    endpoint="weather",
    params={"session_key": sample_session_key}
)

print("Weather shape:", weather_df.shape)
weather_df.head()


Weather shape: (161, 10)


,date,session_key,wind_direction,meeting_key,wind_speed,rainfall,track_temperature,air_temperature,humidity,pressure
0,2023-03-05T14:01:47.286000+00:00,7953,176,1141,1.2,0,35.1,29.8,19.0,1016.5
1,2023-03-05T14:02:47.301000+00:00,7953,182,1141,1.2,0,35.0,29.7,19.0,1016.5
2,2023-03-05T14:03:47.300000+00:00,7953,156,1141,1.1,0,34.9,29.7,19.0,1016.5
3,2023-03-05T14:04:47.314000+00:00,7953,201,1141,0.8,0,34.9,29.6,19.0,1016.5
4,2023-03-05T14:05:47.297000+00:00,7953,219,1141,0.8,0,34.8,29.6,19.0,1016.5


### 4.2 Weather Sanity Checks

In [ ]:
assert not weather_df.empty, "Weather dataframe is empty"

expected_weather_cols = {
    "air_temperature",
    "track_temperature",
    "humidity",
    "wind_speed",
    "wind_direction",
    "date",
}

missing = expected_weather_cols - set(weather_df.columns)
assert not missing, f"Missing expected weather columns: {missing}"

print("Weather sanity checks passed")


Weather sanity checks passed


### 4.3 Normalize Weather Timestamps

In [ ]:
weather_df = normalize_timestamps(
    weather_df,
    ["date"]
)

assert weather_df["date"].dt.tz is not None

print("Weather timestamps normalized")


Weather timestamps normalized


### 4.4 Inspect Weather Sampling Frequency

In [ ]:
weather_df["date"].diff().value_counts().head()


,count
date,
0 days 00:00:59.999000,62
0 days 00:01:00.015000,27
0 days 00:01:00.014000,22
0 days 00:00:59.998000,12
0 days 00:00:59.983000,9


### 4.5 Temporal Alignment Strategy
Weather data is sampled at fixed or semi-fixed intervals, while lap data is event-driven.

For each lap event, weather features will be as-of joined using the most recent past weather observation.

This avoids forward-looking leakage and preserves causality.

## **Section 5** | **First Temporal Join** (Bronze -> Silver)
**Goal**: Create our **first engineered feature table** by aligning lap events with contemporaneous weather without leakage.

### 5.1 Prepare Data for As-Of Join
Requirements of

```
merge_asof:
```
* Both DataFrames sorted by time
* Join key must be named consistently


In [ ]:
# Prepare laps
laps_silver = laps_df.copy()
laps_silver = laps_silver.sort_values("date_start")

# Prepare weather
weather_silver = weather_df.copy()
weather_silver = weather_silver.sort_values("date")

print("Prepared laps and weather for temporal join")


Prepared laps and weather for temporal join


### 5.2 Exectute As-Of Join
We join backward to avoid future leakage.

In [ ]:
laps_weather_df = pd.merge_asof(
    left=laps_silver,
    right=weather_silver,
    left_on="date_start",
    right_on="date",
    direction="backward",
)

print("Lap-weather joined shape:", laps_weather_df.shape)
laps_weather_df.head()


Lap-weather joined shape: (1058, 26)


,meeting_key_x,session_key_x,driver_number,lap_number,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,is_pit_out_lap,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed,date,session_key_y,wind_direction,meeting_key_y,wind_speed,rainfall,track_temperature,air_temperature,humidity,pressure
0,1141,7953,1,1,2023-03-05 15:03:38.500000+00:00,33.103,42.414,23.842,232.0,231.0,False,99.359,"[None, 2048, 2049, 2049, 2051, 2051, 2049, 204...","[2051, 2049, 2051, 2049, 2051, 2049, 2049, 204...","[2051, 2049, 2049, 2049, 2051, 2051]",252.0,2023-03-05 15:02:47.548000+00:00,7953,0,1141,0.6,0,31.3,27.3,22.0,1016.7
1,1141,7953,21,1,2023-03-05 15:03:38.500000+00:00,38.329,46.835,25.179,217.0,226.0,False,110.343,"[None, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]",253.0,2023-03-05 15:02:47.548000+00:00,7953,0,1141,0.6,0,31.3,27.3,22.0,1016.7
2,1141,7953,10,1,2023-03-05 15:03:38.500000+00:00,38.115,46.780,25.135,214.0,234.0,False,110.030,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]",271.0,2023-03-05 15:02:47.548000+00:00,7953,0,1141,0.6,0,31.3,27.3,22.0,1016.7
3,1141,7953,81,1,2023-03-05 15:03:38.500000+00:00,37.748,45.783,24.952,231.0,247.0,False,108.483,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2048]",250.0,2023-03-05 15:02:47.548000+00:00,7953,0,1141,0.6,0,31.3,27.3,22.0,1016.7
4,1141,7953,18,1,2023-03-05 15:03:38.500000+00:00,36.263,44.143,24.609,223.0,250.0,False,105.015,"[None, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]",261.0,2023-03-05 15:02:47.548000+00:00,7953,0,1141,0.6,0,31.3,27.3,22.0,1016.7


### 5.3 Sanity Checks: Leakage & Coverage

In [ ]:
# Weather should not be null for most laps
weather_null_rate = laps_weather_df["air_temperature"].isna().mean()
print("Weather null rate:", weather_null_rate)

assert weather_null_rate < 0.05, "Too much missing weather data"

# Check temporal correctness
assert (
    laps_weather_df["date"] <= laps_weather_df["date_start"]
).all(), "Forward-looking weather leakage detected"

print("Temporal join sanity checks passed")


Weather null rate: 0.0
Temporal join sanity checks passed


### 5.4 Inspect Feature Stability

In [ ]:
laps_weather_df[
    ["air_temperature", "track_temperature", "humidity", "wind_speed"]
].describe()


,air_temperature,track_temperature,humidity,wind_speed
count,1058.000000,1058.000000,1058.000000,1058.000000
mean,26.759452,29.938752,21.111531,0.492911
std,0.323431,0.741825,0.375251,0.253410
min,26.200000,28.700000,20.000000,0.000000
25%,26.500000,29.400000,21.000000,0.400000
50%,26.700000,29.900000,21.000000,0.600000
75%,27.100000,30.500000,21.000000,0.600000
max,27.300000,31.300000,22.000000,1.200000


The Silver layer represents event-level data with validated temporal alignment across sources.

Each row corresponds to a lap event enriched with contemporaneous environmental conditions, suitable for exploratory analysis and downstream aggregation.

## **Section 6** | **Stint-Level Feature Engineering**
**Goal**: Move from raw lap events to **strategy-relevant abstractions** while preserving traceability back to raw data.

### 6.1 Define Stints
A **stint** = continuous laps between  pit exits.
We'dd define it using:
* **driver_number**
* **is_pit_out_lap**

In [ ]:
laps_stint = laps_weather_df.copy()

# Sort correctly
laps_stint = laps_stint.sort_values(
    ["driver_number", "date_start"]
)

# Stint ID: cumulative count of pit-out laps
laps_stint["stint_id"] = (
    laps_stint.groupby("driver_number")["is_pit_out_lap"]
    .cumsum()
)

laps_stint[["driver_number", "lap_number", "is_pit_out_lap", "stint_id"]].head(10)


,driver_number,lap_number,is_pit_out_lap,stint_id
0,1,1,False,0
20,1,2,False,0
40,1,3,False,0
60,1,4,False,0
80,1,5,False,0
100,1,6,False,0
120,1,7,False,0
140,1,8,False,0
160,1,9,False,0
180,1,10,False,0


### 6.2 Stint Sanity Checks

In [ ]:
assert laps_stint["stint_id"].ge(0).all(), "Negative stint IDs detected"

# Each stint should have multiple laps (usually)
stint_sizes = laps_stint.groupby(["driver_number", "stint_id"]).size()
print(stint_sizes.describe())


count    70.000000
mean     15.114286
std       5.839751
min       1.000000
25%      11.000000
50%      15.000000
75%      18.000000
max      29.000000
dtype: float64


### 6.3 Aggregate to Stint-Level Features

In [ ]:
stint_features = (
    laps_stint
    .groupby(["driver_number", "stint_id"])
    .agg(
        stint_laps=("lap_number", "count"),
        avg_lap_time=("lap_duration", "mean"),
        lap_time_std=("lap_duration", "std"),
        avg_track_temp=("track_temperature", "mean"),
        avg_air_temp=("air_temperature", "mean"),
        avg_wind_speed=("wind_speed", "mean"),
    )
    .reset_index()
)

print("Stint features shape:", stint_features.shape)
stint_features.head()

Stint features shape: (70, 8)


,driver_number,stint_id,stint_laps,avg_lap_time,lap_time_std,avg_track_temp,avg_air_temp,avg_wind_speed
0,1,0,14,98.500643,0.887958,30.928571,27.185714,0.714286
1,1,1,22,98.717182,4.450803,29.995455,26.813636,0.590909
2,1,2,21,99.347095,6.935864,29.142857,26.404762,0.261905
3,2,0,12,101.624167,1.962117,30.983333,27.183333,0.666667
4,2,1,18,100.756389,4.930702,30.138889,26.855556,0.588889


### 6.4 Interpretability Check

In [ ]:
stint_features.describe()

,driver_number,stint_id,stint_laps,avg_lap_time,lap_time_std,avg_track_temp,avg_air_temp,avg_wind_speed
count,70.000000,70.000000,70.000000,70.000000,69.000000,70.000000,70.000000,70.000000
mean,24.357143,1.414286,15.114286,101.249269,5.909583,30.022233,26.794860,0.514619
std,20.454995,1.279638,5.839751,3.466681,3.833825,0.741249,0.317381,0.178269
min,1.000000,0.000000,1.000000,98.500643,0.860936,28.750000,26.200000,0.100000
25%,10.250000,0.000000,11.000000,99.523884,2.980650,29.257500,26.460516,0.337778
50%,21.000000,1.000000,15.000000,100.652999,5.543185,30.075000,26.817532,0.590000
75%,27.000000,2.000000,18.000000,101.618025,7.056910,30.896667,27.170238,0.639722
max,81.000000,6.000000,29.000000,124.847000,23.813235,31.066667,27.211111,0.866667


## **Section 7** | **Gold Layer & Canonical Feature Table**
**Goal**: Produce a clean, recruiter-ready table that demonstrates:
* Proper keys
* Clear grain
* Business-relevant abstractions

### 7.1 Canonical Identifier Cleanup

In [ ]:
gold_stint_features = stint_features.copy()

gold_stint_features = gold_stint_features.rename(
    columns={
        "driver_number": "driver_id"
    }
)

# Explicit grain declaration
gold_stint_features["grain"] = "driver_stint"

gold_stint_features.head()


,driver_id,stint_id,stint_laps,avg_lap_time,lap_time_std,avg_track_temp,avg_air_temp,avg_wind_speed,grain
0,1,0,14,98.500643,0.887958,30.928571,27.185714,0.714286,driver_stint
1,1,1,22,98.717182,4.450803,29.995455,26.813636,0.590909,driver_stint
2,1,2,21,99.347095,6.935864,29.142857,26.404762,0.261905,driver_stint
3,2,0,12,101.624167,1.962117,30.983333,27.183333,0.666667,driver_stint
4,2,1,18,100.756389,4.930702,30.138889,26.855556,0.588889,driver_stint


###7.2 Add Session Context

In [ ]:
session_context = race_sessions_df[
    ["session_key", "meeting_key", "country_name"]
].drop_duplicates()

# Since we only processed one session, this is safe
gold_stint_features["session_key"] = sample_session_key

gold_stint_features = gold_stint_features.merge(
    session_context,
    on="session_key",
    how="left"
)

gold_stint_features.head()


,driver_id,stint_id,stint_laps,avg_lap_time,lap_time_std,avg_track_temp,avg_air_temp,avg_wind_speed,grain,session_key,meeting_key,country_name
0,1,0,14,98.500643,0.887958,30.928571,27.185714,0.714286,driver_stint,7953,1141,Bahrain
1,1,1,22,98.717182,4.450803,29.995455,26.813636,0.590909,driver_stint,7953,1141,Bahrain
2,1,2,21,99.347095,6.935864,29.142857,26.404762,0.261905,driver_stint,7953,1141,Bahrain
3,2,0,12,101.624167,1.962117,30.983333,27.183333,0.666667,driver_stint,7953,1141,Bahrain
4,2,1,18,100.756389,4.930702,30.138889,26.855556,0.588889,driver_stint,7953,1141,Bahrain


### 7.3 Final Sanity Checks

In [ ]:
assert gold_stint_features.notna()["avg_lap_time"].all()
assert gold_stint_features["stint_laps"].ge(1).all()

print("Gold layer sanity checks passed")


Gold layer sanity checks passed


### 7.4 Save Artifact

In [ ]:
gold_stint_features.to_csv(
    "gold_stint_features.csv",
    index=False
)

print("Gold layer exported")


Gold layer exported
